In [1]:
from typing import List, Sequence
import asyncio
import nest_asyncio
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
nest_asyncio.apply()

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [3]:
zuschauer = UserProxyAgent("Zuschauer")

redakteur = AssistantAgent(
    name="Redakteur",
    description="An- und Abmoderation, Überleitungen zwischen den Themen",
    system_message = """
        Du bist Haupt-Moderator einer Tagesschau ählichen Nachrichten-Sendung namens BTB-Schau.
        In deinem Team befinden sich Inland- und Ausland-Korrespondennten.
        Sprich nur deinen Teil, mach keine Effekte sonder nur deinen Sprech-Teil.
        Außerdem gibt es eine Regie, die diese dann Zuschaltet, sobald sie es für richtig hält. Du musst dich nicht darum kümmern.
        Du bist vor allem für An- und Abmoderation und für den Austausch mit den Korrespondenten zuständig.
    """,
    model_client=model_client,
)

inland = AssistantAgent(
    name="Inland",
    description="Korrespondent für Inland-Themen.",
    system_message = """
        Du bist der Inland-Korrespondent einer Tagesschau ähnlichen Sendung.
        Du Kommunizierst mit dem Redakteur und wirst nur bei Inland-Themen von der Regie eingeschaltet.
        Du kennst diese Themen und wenn nicht überlegst du dir einfach was dazu, es wird nicht gefact-checked.
    """,
    model_client=model_client,
)

ausland = AssistantAgent(
    name="Ausland",
    description="Korrespondent für Ausland-Themen.",
    system_message = """
        Du bist der Auslands-Korrespondent einer Tagesschau ähnlichen Sendung.
        Du Kommunizierst mit dem Redakteur und wirst nur bei Ausland-Themen von der Regie eingeschaltet.
        Du kennst diese Themen und wenn nicht überlegst du dir einfach was dazu, es wird nicht gefact-checked.
    """,
    model_client=model_client,
)

In [1]:


groupchat = SelectorGroupChat(
    participants=[redakteur, inland, ausland],
    model_client=model_client,
    selector_prompt = """
        Du bist die Regie der Tagesschau und steuerst, wer reden kann.
        Der Redakteur macht die An- und Abmoderation und die Übergänge zwischen den Themen.
        Zu den Themen wird jeweils immer der Auslands- oder der Inland-Korrespondent zugeschaltet.

        Der Ablauf ist so: Redakteur -> Korrespondent -> Redakteur -> Next Topic

        Wähl einen Agenten aus:
        {roles}

        Transkript bis jetzt:
        {history}

    """,
    termination_condition=TextMentionTermination("CUT"),
    allow_repeated_speaker=True
)

NameError: name 'SelectorGroupChat' is not defined

In [2]:

await Console(groupchat.run_stream(task="Starte eine Episoder der BTB-Schau"))

await model_client.close()

NameError: name 'Console' is not defined